### Import

### Business Logic

In [ ]:
# Business Logic
# =============================================================================
def func_parse_data(data: dict, **config) -> dict:
    return {"company": data["raw_text"], "base_score": 50}

def func_quant_analysis(data: dict, **config) -> dict:
    # 정량 분석: 점수 +10
    return {"company": data["company"], "quant_score": data["base_score"] + 10, "source": "A"}

def func_qual_analysis(data: dict, **config) -> dict:
    # 정성 분석: 점수 +20
    return {"company": data["company"], "qual_score": data["base_score"] + 20, "source": "B"}

def func_merge_decision(data: dict, **config) -> dict:
    # [Merge Logic] 두 결과를 받아서 최종 판단
    inputs = data.get("__inputs__", [])
    total_score = 0
    details = []
    
    for item in inputs:
        s_quant = item.get("quant_score", 0)
        s_qual = item.get("qual_score", 0)
        total_score += s_quant + s_qual
        details.append(item.get("source"))

    decision = "APPROVE" if total_score > 100 else "REJECT"
    return {
        "final_score": total_score,
        "decision": decision,
        "merged_sources": details
    }




### Main

In [ ]:

# =============================================================================
# [4] Main Execution (The Driver)
# =============================================================================


# 1. Setup Engine & Validator
validator = 
engine = CSPNEngine(validator)

# 함수 등록 (Mocking importlib)
engine.register_function("parse", func_parse_data)
engine.register_function("quant", func_quant_analysis)
engine.register_function("qual", func_qual_analysis)
engine.register_function("merge", func_merge_decision)

# 2. Define Process Topology
process = CSPNProcess("RISK_PROCESS_V1")

# Start Node
t_start = TaskSpec(
    task_id="START", target="parse", 
    input_spec_id="RAW", output_spec_id="PARSED"
)
# Branch 1
t_a = TaskSpec(
    task_id="TASK_A", target="quant", 
    input_spec_id="PARSED", output_spec_id="RES_A"
)
# Branch 2
t_b = TaskSpec(
    task_id="TASK_B", target="qual", 
    input_spec_id="PARSED", output_spec_id="RES_B"
)
# Merge Node (CUSTOM Strategy 필수!)
t_c = TaskSpec(
    task_id="TASK_C", target="merge", 
    input_spec_id="ANY", output_spec_id="FINAL",
    merge_strategy=MergeStrategy.CUSTOM 
)

process.add_task(t_start)
process.add_task(t_a)
process.add_task(t_b)
process.add_task(t_c)

# Links: Start -> [A, B] -> C
process.add_link("START", "TASK_A")
process.add_link("START", "TASK_B")
process.add_link("TASK_A", "TASK_C")
process.add_link("TASK_B", "TASK_C")

# 3. Inject Initial Token
initial_token = Token(
    trace_id="TR-2026-001",
    content={"raw_text": "SAMSUNG_ELEC"}
)
process.token_queue.append(("START", initial_token))

print("\n>>> [Run] Start Loop")
step_count = 0
while process.token_queue:
    step_count += 1
    print(f"\n--- Step {step_count} ---")
    
    result = engine.run_step(process)
    
    if result and result.success:
        print(f"[Engine] Task '{result.task_id}' Finished.")
    else:
        print("[Engine] Idle or Failed.")

# 4. Final Result Check
print("\n>>> [System] Workflow Finished.")
if process.completed_tokens:
    final = process.completed_tokens[0]
    print(f"Final Output: {final.content}")
    print(f"History Trace: {final.history}")
else:
    print("No tokens reached end.")